# Spatial Dropout (Dropout2d) — Temelden Derine (CNN)

Bu notebook şunları öğretir:
- Spatial Dropout nedir?
- Klasik Dropout’tan farkı ne?
- Neden CNN’lerde işe yarar?
- Matematiksel bakış: maske nasıl uygulanır?
- PyTorch implementasyonu (nn.Dropout2d / custom)
- DropBlock ile farkı
- Pratik ayarlar, nerede konur, hangi oran mantıklı?


## 1) Dropout ailesi: neyi düzenliyoruz?

**Regularization**: modelin training set’e aşırı uyumunu (overfitting) azaltmak.

Dropout’un temel fikri:
- Eğitim sırasında bazı aktivasyonları rastgele sıfırla
- Model "tek bir sinyale bağımlı" kalmasın
- Bir nevi *ensemble* etkisi yarat (farklı alt-ağlar)

Ama CNN’de bir problem var:
- Bir feature map (kanal) içindeki pikseller **çok koreledir**.
- Klasik dropout (element-wise) rastgele pikselleri silince etkisi zayıflar.

Bu yüzden CNN’e daha uygun dropout türevleri gelir:
- **Spatial Dropout / Dropout2d**: kanalı komple düşür
- **DropBlock**: uzamsal blok düşür
- **DropPath / Stochastic Depth**: entire path / block düşür


## 2) Spatial Dropout nedir?

Spatial Dropout (PyTorch'ta pratikte **`nn.Dropout2d`**):
- Giriş tensörü shape'i: **[B, C, H, W]**
- Her örnek (B) için her kanal (C) ya **tamamen korunur** ya da **tamamen sıfırlanır**.
- Maske shape'i: **[B, C, 1, 1]** (sonra H,W’ye broadcast edilir)

Yani:
- Klasik dropout: `mask ~ Bernoulli(keep)` shape `[B,C,H,W]`
- Spatial dropout: `mask ~ Bernoulli(keep)` shape `[B,C,1,1]`

👉 Önemli: Spatial Dropout **spatial alanı (H,W) değil**, **kanalları** hedefler.
İsim yanıltmasın: "spatial" burada CNN feature map mantığından gelir.


## 3) Amaç ne?

CNN’de her kanal genelde bir pattern yakalar:
- edge / texture / orientation / local structure gibi

Model bazen birkaç kanala aşırı yaslanır.
**Spatial Dropout** der ki:
> "Bazı kanalları komple kapatıyorum, diğer kanallardan da aynı işi öğren."

Bu şunları sağlar:
- Kanal co-adaptation azalır (kanalların birbirine aşırı bağımlılığı kırılır)
- Daha robust feature öğrenimi
- Küçük veri / domain shift olan senaryolarda fayda


## 4) Matematiksel formül (basit ve net)

Girdi:  
$$x \in \mathbb{R}^{B \times C \times H \times W}$$

Keep olasılığı: $q = 1 - p$  
Maske:  
$$m \sim \text{Bernoulli}(q) \quad , \quad m \in \{0,1\}^{B \times C \times 1 \times 1}$$

Uygulama (inverted dropout):
$$y = \frac{x \odot m}{q}$$

Böylece training sırasında beklenen değer korunur:
$$\mathbb{E}[y] = x$$

Eval modunda dropout kapalıdır:
$$y = x$$


In [1]:
import torch
import torch.nn as nn

torch.manual_seed(0)

B, C, H, W = 2, 4, 3, 3
x = torch.randn(B, C, H, W)
x

tensor([[[[-1.1258, -1.1524, -0.2506],
          [-0.4339,  0.8487,  0.6920],
          [-0.3160, -2.1152,  0.3223]],

         [[-1.2633,  0.3500,  0.3081],
          [ 0.1198,  1.2377,  1.1168],
          [-0.2473, -1.3527, -1.6959]],

         [[ 0.5667,  0.7935,  0.5988],
          [-1.5551, -0.3414,  1.8530],
          [ 0.7502, -0.5855, -0.1734]],

         [[ 0.1835,  1.3894,  1.5863],
          [ 0.9463, -0.8437, -0.6136],
          [ 0.0316, -0.4927,  0.2484]]],


        [[[ 0.4397,  0.1124,  0.6408],
          [ 0.4412, -0.1023,  0.7924],
          [-0.2897,  0.0525,  0.5229]],

         [[ 2.3022, -1.4689, -1.5867],
          [-0.6731,  0.8728,  1.0554],
          [ 0.1778, -0.2303, -0.3918]],

         [[ 0.5433, -0.3952,  1.5091],
          [ 2.0820,  1.7067,  2.3804],
          [-1.1256, -0.3170, -1.0925]],

         [[-0.0852,  0.3276, -0.7607],
          [-1.5991,  0.0185, -0.7504],
          [ 0.1854,  0.6211,  0.6382]]]])

## 5) PyTorch’ta hazır kullanım: nn.Dropout2d

PyTorch’ta **`nn.Dropout2d`** şunu yapar:
- (N, C, H, W) girişte **kanal bazlı** maske uygular
- Her kanal için tüm (H,W) birlikte sıfırlanır

Not:
- PyTorch dokümantasyonunda `Dropout2d` bazen "feature map dropout" diye geçer.
- Mantık: *channel-wise dropout*.


In [2]:
drop2d = nn.Dropout2d(p=0.5)
drop2d.train()

y = drop2d(x)
y

tensor([[[[-2.2517, -2.3047, -0.5012],
          [-0.8678,  1.6974,  1.3840],
          [-0.6320, -4.2304,  0.6445]],

         [[-2.5267,  0.7000,  0.6163],
          [ 0.2397,  2.4753,  2.2336],
          [-0.4946, -2.7053, -3.3919]],

         [[ 0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000]]],


        [[[ 0.8794,  0.2248,  1.2816],
          [ 0.8823, -0.2046,  1.5849],
          [-0.5793,  0.1050,  1.0457]],

         [[ 0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000]],

         [[ 0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000]],

         [[-0.1704,  0.6552, -1.5214],
          [-3.1982,  0.0370, -1.5009],
          [ 0.3708,  1.2423,  1.2764]]]])

In [3]:
# Hangi kanallar sıfırlandı görelim (örnek 0 için)
# Bir kanal tamamen 0 ise mean(abs)=0 çıkar.
channel_energy = y[0].abs().mean(dim=(1,2))
channel_energy

tensor([1.6126, 1.7092, 0.0000, 0.0000])

Yukarıdaki `channel_energy` değerlerinde **0 olan kanallar** komple düşmüş demektir.

Ayrıca şu kritik noktayı gör:
- Dropout2d training'de aktif
- eval'de pasif


In [4]:
drop2d.eval()
y_eval = drop2d(x)
torch.allclose(x, y_eval)

True

## 6) Custom Spatial Dropout implementasyonu (tam kontrol)

Bazen şunları istersin:
- maske üretimini görüp debug etmek
- farklı scaling stratejisi
- sadece belirli katmanlara uygulamak

Aşağıdaki modül birebir şu formülü uygular:
$$y = x \odot m / q$$
burada $m$ shape `[B,C,1,1]`.


In [5]:
class SpatialDropout2D(nn.Module):
    def __init__(self, p: float = 0.5):
        super().__init__()
        if not (0.0 <= p < 1.0):
            raise ValueError("p must be in [0,1).")
        self.p = float(p)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if (not self.training) or self.p == 0.0:
            return x
        q = 1.0 - self.p
        # mask: [B, C, 1, 1]
        mask = torch.empty((x.size(0), x.size(1), 1, 1), device=x.device, dtype=x.dtype).bernoulli_(q)
        return x * mask / q

sd = SpatialDropout2D(p=0.5)
sd.train()
y2 = sd(x)
y2

tensor([[[[-2.2517, -2.3047, -0.5012],
          [-0.8678,  1.6974,  1.3840],
          [-0.6320, -4.2304,  0.6445]],

         [[-2.5267,  0.7000,  0.6163],
          [ 0.2397,  2.4753,  2.2336],
          [-0.4946, -2.7053, -3.3919]],

         [[ 0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000]]],


        [[[ 0.8794,  0.2248,  1.2816],
          [ 0.8823, -0.2046,  1.5849],
          [-0.5793,  0.1050,  1.0457]],

         [[ 4.6044, -2.9378, -3.1734],
          [-1.3462,  1.7457,  2.1107],
          [ 0.3557, -0.4607, -0.7835]],

         [[ 1.0866, -0.7903,  3.0182],
          [ 4.1639,  3.4134,  4.7607],
          [-2.2512, -0.6340, -2.1849]],

         [[-0.0000,  0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000]]]])

## 7) Nerede kullanılır? (CNN blok akışı)

Genel pratik:
- Conv → Norm → Activation → **Spatial Dropout**

Örnek:
- Conv → BN → ReLU → Dropout2d

Residual blokta:
- Ana dalın sonunda veya attention sonrası

Dikkat:
- Çok erken katmanlarda (inputa çok yakın) aşırı dropout detay kaybı yaratabilir.


## 8) DropBlock ile farkı (çok net)

- **Spatial Dropout**: Kanalı komple kapatır (C ekseninde).
- **DropBlock**: Aynı kanal içinde H×W üzerinde *blok* kapatır.

Ne zaman hangisi?
- Kanal bağımlılığı problemi → Spatial Dropout
- Lokal bölgeye aşırı güvenme / spatial korelasyon → DropBlock


## 9) Pratik ayarlar (sahada iş gören)

- p genelde: **0.05 – 0.3**
- Çok derin/overfit modellerde: **0.2 – 0.4** denenebilir
- BN/SyncBN varsa dropout oranını abartma (özellikle küçük batch)
- Dropout2d’yi genelde **activation sonrası** koy

İpucu:
- Eğer zaten güçlü augmentation + weight decay + DropPath varsa,
  Spatial Dropout’u düşük tut (p≈0.05–0.15).


## 10) Mini örnek blok

Aşağıdaki blok: Conv → BN → SiLU → Dropout2d.
Gerçek projede bunu residual/attention bloklarının içine entegre edersin.


In [6]:
class ConvBNActSD(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1, act="silu", sd_p=0.1):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.SiLU(inplace=True) if act == "silu" else nn.ReLU(inplace=True)
        self.sd = nn.Dropout2d(p=sd_p)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        x = self.sd(x)
        return x

blk = ConvBNActSD(3, 16, sd_p=0.2)
blk.train()
out = blk(torch.randn(2,3,32,32))
out.shape

torch.Size([2, 16, 32, 32])

## 11) Checklist (senin repo için)

- [ ] `nn.Dropout2d` ile basit entegrasyon
- [ ] Custom `SpatialDropout2D` ile maskeyi doğrulama
- [ ] Ablation: p=0.0 / 0.1 / 0.2
- [ ] DropBlock + DropPath ile birlikteyken p düşür

Bittiğinde sıradaki konu: **Stochastic Depth**.
